## Final Group Review

> so... 12 weeks have passed. How have we wasted these 12 weeks?

## Addition 1 - 2 Player

In the beginning, we added a two player system to our End level.
This was done with one file by delegating movement inputs down to the instances of the Player class.

In [ ]:
if (this.pressedKeys[this.keypress.up] && this.pressedKeys[this.keypress.left]) {
    this.velocity.y -= this.yVelocity;
    this.velocity.x -= this.xVelocity;
    this.direction = 'upLeft';
} else if (this.pressedKeys[this.keypress.up] && this.pressedKeys[this.keypress.right]) {
    this.velocity.y -= this.yVelocity;
    this.velocity.x += this.xVelocity;
    this.direction = 'upRight';

a short snippet of the movement code using instanced variables

## Addition 2 - Enemy

The Enderman, or enemy, detects  the nearest player and paths towards them. It also kills on contact and uses death particle effects.

![enderman](https://ibb.co/5dWvjbn)

This part of the Enderman catalogues Players into a list, calculates distance to player, and if the player is within the Enderman's minimum distance sets it as "nearest," or as its target.

In [ ]:
const players = this.gameEnv.gameObjects.filter(obj => 
    obj.constructor.name === 'Player'
);

if (players.length === 0) return;

// Find nearest player
let nearest = players[0];
let minDist = Infinity;

for (const player of players) {
    const dx = player.position.x - this.position.x;
    const dy = player.position.y - this.position.y;
    const dist = Math.sqrt(dx*dx + dy*dy);
    if (dist < minDist) {
        minDist = dist;
        nearest = player;
    }
}

This part of the code sets the Endermans speed (as an adjustable variable) and resets the direction x and direction y according to the "nearest" player set above. It also calculates the angle as well as updates the Enderman's position continuously

In [ ]:
const speed = 1.5; // Adjust speed as needed
const dx = nearest.position.x - this.position.x;
const dy = nearest.position.y - this.position.y;
const angle = Math.atan2(dy, dx);

// Update position
this.position.x += Math.cos(angle) * speed;
this.position.y += Math.sin(angle) * speed;

Here, we check for collision by setting a hitbox for the player and the Enderman

In [ ]:
// Check for collision with any player
for (const player of players) {
    // Calculate distance for hitbox collision
    const playerX = player.position.x + player.width / 2;
    const playerY = player.position.y + player.height / 2;
    const enemyX = this.position.x + this.width / 2;
    const enemyY = this.position.y + this.height / 2;
    
    const dx = playerX - enemyX;
    const dy = playerY - enemyY;
    const distance = Math.sqrt(dx*dx + dy*dy);
    
    // Hitbox collision - adjust values as needed
    const collisionThreshold = (player.width * player.hitbox.widthPercentage + 
                            this.width * this.hitbox.widthPercentage) / 2;
    
    if (distance < collisionThreshold) {
        // Set killing flag to prevent repeated kills
        this.isKilling = true;

full code below

In [ ]:
const sprite_src_enemy = path + "/images/gamify/enderman.png";
const sprite_data_enemy = {
    id: 'Enderman',
    greeting: "You feel a dark presence...",
    src: sprite_src_enemy,
    SCALE_FACTOR: 10,
    ANIMATION_RATE: 50,
    pixels: {height: 1504, width: 574},
    INIT_POSITION: { x: width / 2, y: height / 4 },
    orientation: {rows: 1, columns: 1},
    down: {row: 0, start: 0, columns: 1},
    hitbox: { widthPercentage: 0.4, heightPercentage: 0.4 },
    zIndex: 10,
    isKilling: false, // Flag to prevent multiple kills
    
    // The update method with all functionality inline
    update: function() {
        // Skip update if already in killing process
        if (this.isKilling) {
            return;
        }
        
        // Find all player objects
        const players = this.gameEnv.gameObjects.filter(obj => 
            obj.constructor.name === 'Player'
        );
        
        if (players.length === 0) return;
        
        // Find nearest player
        let nearest = players[0];
        let minDist = Infinity;

        for (const player of players) {
            const dx = player.position.x - this.position.x;
            const dy = player.position.y - this.position.y;
            const dist = Math.sqrt(dx*dx + dy*dy);
            if (dist < minDist) {
                minDist = dist;
                nearest = player;
            }
        }

        // Move towards nearest player
        const speed = 1.5; // Adjust speed as needed
        const dx = nearest.position.x - this.position.x;
        const dy = nearest.position.y - this.position.y;
        const angle = Math.atan2(dy, dx);
        
        // Update position
        this.position.x += Math.cos(angle) * speed;
        this.position.y += Math.sin(angle) * speed;
        
        // Check for collision with any player
        for (const player of players) {
            // Calculate distance for hitbox collision
            const playerX = player.position.x + player.width / 2;
            const playerY = player.position.y + player.height / 2;
            const enemyX = this.position.x + this.width / 2;
            const enemyY = this.position.y + this.height / 2;
            
            const dx = playerX - enemyX;
            const dy = playerY - enemyY;
            const distance = Math.sqrt(dx*dx + dy*dy);
            
            // Hitbox collision - adjust values as needed
            const collisionThreshold = (player.width * player.hitbox.widthPercentage + 
                                    this.width * this.hitbox.widthPercentage) / 2;
            
            if (distance < collisionThreshold) {
                // Set killing flag to prevent repeated kills
                this.isKilling = true;
                
                // === PLAYER DEATH: ALL FUNCTIONALITY INLINE ===
                
                // 1. Play death animation - particle effect
                const playerX = player.position.x;
                const playerY = player.position.y;
                
                // Create explosion effect
                for (let i = 0; i < 20; i++) {
                    const particle = document.createElement('div');
                    particle.style.position = 'absolute';
                    particle.style.width = '5px';
                    particle.style.height = '5px';
                    particle.style.backgroundColor = 'red';
                    particle.style.left = `${playerX + player.width/2}px`;
                    particle.style.top = `${playerY + player.height/2}px`;
                    particle.style.zIndex = '9999';
                    document.body.appendChild(particle);
                    
                    // Animate particles outward
                    const angle = Math.random() * Math.PI * 2;
                    const speed = Math.random() * 5 + 2;
                    const distance = Math.random() * 100 + 50;
                    
                    const destX = Math.cos(angle) * distance;
                    const destY = Math.sin(angle) * distance;
                    
                    particle.animate(
                        [
                            { transform: 'translate(0, 0)', opacity: 1 },
                            { transform: `translate(${destX}px, ${destY}px)`, opacity: 0 }
                        ],
                        {
                            duration: 1000,
                            easing: 'ease-out',
                            fill: 'forwards'
                        }
                    );
                    
                    // Remove particle after animation
                    setTimeout(() => {
                        if (particle.parentNode) {
                            particle.parentNode.removeChild(particle);
                        }
                    }, 1000);
                }
                
                // 2. Show death message dialog
                const deathMessage = document.createElement('div');
                deathMessage.style.position = 'fixed';
                deathMessage.style.top = '50%';
                deathMessage.style.left = '50%';
                deathMessage.style.transform = 'translate(-50%, -50%)';
                deathMessage.style.backgroundColor = 'rgba(0, 0, 0, 0.8)';
                deathMessage.style.color = '#FF0000';
                deathMessage.style.padding = '30px';
                deathMessage.style.borderRadius = '10px';
                deathMessage.style.fontFamily = "'Press Start 2P', sans-serif";
                deathMessage.style.fontSize = '24px';
                deathMessage.style.textAlign = 'center';
                deathMessage.style.zIndex = '10000';
                deathMessage.style.border = '3px solid #FF0000';
                deathMessage.style.boxShadow = '0 0 20px rgba(255, 0, 0, 0.5)';
                deathMessage.style.width = '400px';
                deathMessage.innerHTML = `
                    <div style="margin-bottom: 20px;">☠️ YOU DIED ☠️</div>
                    <div style="font-size: 16px; margin-bottom: 20px;">The Enderman got you!</div>
                    <div style="font-size: 14px;">Respawning in 2 seconds...</div>
                `;
                
                document.body.appendChild(deathMessage);
                
                // Remove message after delay
                setTimeout(() => {
                    if (deathMessage.parentNode) {
                        deathMessage.parentNode.removeChild(deathMessage);
                    }
                }, 2000);
                
                // 3. Reset the level after a short delay using page reload for reliability
                setTimeout(() => {
                    // Clean up any lingering resources before reload
                    if (self && self.timerInterval) {
                        clearInterval(self.timerInterval);
                    }
                    
                    // Force a complete page reload - most reliable way to reset
                    location.reload();
                }, 2000); // 2 second delay before reset
                
                break;
            }
        }
    }
};

## Addition 3 - Revamped Dialogue

Our new dialogue system not only looks better but keeps immersion in the game as it does not use browser alerts to display dialogue.

- We created 2 new files: DialogBox.js and DialogueSystem.js
- DialogBox.js handles the creation of the ingame box and the sound effect
- DialogueSystem.js handles the usage of DialogBox.js, the creation of the character image, and also the method which the npcs' use this system



implementation in an NPC below:

In [ ]:
const dialogueSystem = this.dialogueSystem;
const sprite_data_endship = {
    // sprite data abbreviated
    dialogues: [
      "The end ship looms before you...",
      "The end ship seems to beckon you to loot the treasure within...",
      "funny purple spaceship heheheheheh",
      "Press 'M' to enter the ship's adventure...",
    ],
    reaction: function() {
      //silent reaction for interaction to work
    },
    interact: function() {
      dialogueSystem.showRandomDialogue(); // Using Dialogue system instead of alert
      // Add event listener for 'm' key press during interaction
      const handleKeyPress = (event) => {
        if (event.key.toLowerCase() === 'e') {
          // Remove the event listener to prevent multiple bindings
          document.removeEventListener('keydown', handleKeyPress);

in the sprite data, there are three main components.
- First, the "dialogues:" list contains all of the possible lines that can be said by the NPC
- Second, the "reaction:" is set to an empty function in order to use the new dialogue system and not show an alert
- Third, the "interact:" uses the random dialogue system from our files to display the text

## Addition 4 - 